# **HISTORICAL DATA**

## **LIBRARIES**

In [ ]:
!pip install requests
!pip install nest_asyncio
!pip install ib_insync

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import yfinance as yf
import nest_asyncio
from ib_insync import *
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

## **API CONNEXION**

In [ ]:
# Interactive Broker API

# Connect to IB Gateway via ngrok's forwarding address
# ngrok config add-authtoken 2rw2xcnDr9K436GIMIPrJDYXrZk_4DYJZX5ic7odKvK27xU3s
# ngrok tcp 4002 (Watch the right address on IB Gateway)
# Replace '4.tcp.eu.ngrok.io' and '16234' with your ngrok forwarding details

# Apply patch for Colab's event loop issue
nest_asyncio.apply()

# Create an IB instance
ib = IB()

try:
    ib.connect('5.tcp.eu.ngrok.io', 12708, clientId=1)
    print("Connected to IB Gateway!")
except Exception as e:
    print(f"Failed to connect: {e}")

## **INTRADAY STOCKS PRICES**

### **FUNCTIONS**

#### **SP500 TICKERS**

In [10]:
def get_sp500_tickers():
    """
    Fetch the list of S&P 500 tickers from Wikipedia.

    Returns:
        pd.DataFrame: DataFrame containing tickers and company names.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    sp500_table = pd.read_html(url)[0]

    return sp500_table[["Symbol", "Security"]]

#### **INTRADAY STOCK PRICES**

In [11]:
def fetch_intraday_prices_ib(ib, symbol, start_date, end_date):
    """
    Fetch 1-minute intraday prices from Interactive Brokers for a given stock over multiple days.

    Args:
        ib (IB): Connected IB instance.
        symbol (str): Stock ticker symbol (e.g., "AAPL").
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.

    Returns:
        pd.DataFrame: DataFrame with timestamp, price, and ticker.
    """
    # Define stock contract
    contract = Stock(symbol, 'SMART', 'USD')

    # Convert dates to datetime
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")

    # Initialize an empty DataFrame to store data
    all_data = pd.DataFrame()

    # Loop through each day in the date range (IB fetches past data)
    current_dt = end_dt
    while current_dt >= start_dt:
        # Ensure correct IB endDateTime format (YYYYMMDD-HH:MM:SS for UTC)
        endDateTime = current_dt.strftime("%Y%m%d-10:05:59")

        # Request historical intraday data (1-minute bars)
        bars = ib.reqHistoricalData(
            contract,
            endDateTime=endDateTime,
            durationStr='1 D',
            barSizeSetting='1 min',
            whatToShow='MIDPOINT',
            useRTH=True,  # Use regular trading hours only
            formatDate=1
        )

        # Convert bars to DataFrame
        df = util.df(bars) if bars else None

        if df is not None and not df.empty:
            df['timestamp'] = pd.to_datetime(df['date'])  # Convert to datetime
            df['ticker'] = symbol  # Add ticker column
            all_data = pd.concat([df, all_data])  # Append data
        else:
            print(f"No data for {symbol} on {current_dt.strftime('%Y-%m-%d')}")

        # Move to the previous day
        current_dt -= timedelta(days=1)

    return all_data.reset_index(drop=True)

### **EXECUTION**

In [12]:
# Get S&P 500 tickers
sp500_tickers = get_sp500_tickers()
tickers = sp500_tickers["Symbol"].tolist()

In [13]:
# Define dates
start_date = (datetime.now() - timedelta(days=2*31)).strftime('%Y-%m-%d')
end_date = datetime.now() .strftime('%Y-%m-%d')

In [ ]:
# Fetch stocks prices
df_stock_price = pd.DataFrame()

for ticker in tickers: # [:200]adjust for the number of ticker you want cause really time consuming

  # Fetch prices
  df = fetch_intraday_prices_ib(ib, ticker, start_date, end_date)

  if 'close' in df.columns :
    # Full missing data
    df = df.ffill().bfill()

    # Select close price and timestamp columns
    df = df.loc[:, ["close", "timestamp"]]
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Filter datetimes
    required_times = [datetime.strptime("09:35", "%H:%M").time(),
                      datetime.strptime("10:00", "%H:%M").time(),
                      datetime.strptime("10:30", "%H:%M").time(),
                      datetime.strptime("11:00", "%H:%M").time(),
                      datetime.strptime("11:30", "%H:%M").time(),
                      datetime.strptime("12:00", "%H:%M").time(),
                      datetime.strptime("12:30", "%H:%M").time(),
                      datetime.strptime("13:00", "%H:%M").time(),
                      datetime.strptime("13:30", "%H:%M").time(),
                      datetime.strptime("14:00", "%H:%M").time(),
                      datetime.strptime("14:30", "%H:%M").time(),
                      datetime.strptime("15:00", "%H:%M").time(),
                      datetime.strptime("15:30", "%H:%M").time(),
                      datetime.strptime("16:00", "%H:%M").time()
                      ]
    df = df[df['timestamp'].dt.time.isin(required_times)]

    # Rearrange dataframe
    df.set_index('timestamp', inplace=True)
    df.loc[:, ticker] = df.pop('close')

    # Add ticker's intraday prices in the dataframe
    df_stock_price = pd.concat([df, df_stock_price], axis=1)
    print(f"Fetched data for {ticker}")

  else :
    print(f"No data for {ticker}")

## **INTRADAY OPTIONS PRICES**

### **FUNCTIONS**

#### **OPTION CHAINS**

In [15]:
def fetch_option_chain(ib, symbol, stock_price, current_date):
    """
    Fetch the option chain for a given ticker symbol using IB API.

    Args:
        ib (IB instance): The IBKR connection object.
        symbol (str): The stock ticker symbol (e.g., "AAPL").
        stock_price (float): The current stock price.
        current_date (str): The current date in 'YYYY-MM-DD' format.

    Returns:
        dict: Filtered option chain with valid expirations and closest strike {"expirations":[exp], "closest_strike":strike}.
    """

    # Qualify the stock contract
    try:
        contracts = ib.qualifyContracts(Stock(symbol, 'SMART', 'USD'))
        if not contracts:
            print(f"Error: No valid contract found for {symbol}")
            return None  # Return None to prevent IndexError
        stock = contracts[0]  # Retrieve the first qualified contract
    except Exception as e:
        print(f"Error qualifying contract for {symbol}: {e}")
        return None

    # Request the option chain
    try:
        option_chain = ib.reqSecDefOptParams(symbol, '', 'STK', stock.conId)
        if not option_chain:
            print(f"Error: No option chain data available for {symbol}")
            return None

        # Extract relevant data
        expirations = sorted(option_chain[0].expirations)
        strikes = sorted(option_chain[0].strikes)

        # Ensure current_date is a datetime object
        current_date = datetime.strptime(current_date, '%Y-%m-%d')

        # Filter expirations between 30 and 180 days
        filtered_expirations = [
            exp for exp in expirations
            if 30 <= (datetime.strptime(exp, '%Y%m%d') - current_date).days <= 180
        ]

        # Avoid fractional strikes to find contracts with IB
        valid_strikes = [s for s in strikes if s % 1 == 0]

        # Find the closest ATM strike
        closest_strike = min(valid_strikes, key=lambda x: abs(x - stock_price)) if valid_strikes else None

        return {'expirations': filtered_expirations, 'closest_strike': closest_strike}

    except Exception as e:
        print(f"Error fetching option chain for {symbol}: {e}")
        return None


#### **OPTION PRICES**

In [16]:
def fetch_intraday_straddle_prices(ib, ticker, expiration, strike, date):
    """
    Fetch intraday option prices from IBKR at 9:35 AM and 10:00 AM US Eastern Time.

    Args:
        ib (IB instance): The IBKR connection object.
        ticker (str): Underlying stock ticker (e.g., 'MMM').
        expiration (str): Expiration date in 'YYYYMMDD' format (e.g., '20250228').
        strike (float): Strike price (e.g., 150.0).
        date (datetime): Date for which to fetch the price.

    Returns:
        int: straddle price for the contract
    """

    # Format Strike Price Properly
    formatted_strike = "{:.8f}".format(strike).rstrip("0").rstrip(".")

    # Create IB Contracts for Call and Put
    call_contract = Option(ticker, expiration, formatted_strike, 'C', 'SMART')
    put_contract = Option(ticker, expiration, formatted_strike, 'P', 'SMART')

    # Request Historical Data
    call_bars = ib.reqHistoricalData(call_contract, date, '1 D', '1 min', 'MIDPOINT', useRTH=True)
    put_bars = ib.reqHistoricalData(put_contract, date, '1 D', '1 min', 'MIDPOINT', useRTH=True)

    # Convert Data to DataFrame
    call_df = pd.DataFrame(call_bars) if call_bars else pd.DataFrame()
    put_df = pd.DataFrame(put_bars) if put_bars else pd.DataFrame()

    # Check if Data Exists
    if call_df.empty or put_df.empty:
        print(f"Error: No data retrieved for {ticker}, expiration {expiration}, strike {formatted_strike}")
        return None

    # Find Closest Available Prices call_df['date'].iloc[0].replace(hour=9, minute=35)).abs().argsort()[:1]
    call_price = call_df.iloc[(call_df['date'] - date).abs().argsort()[:1]]['close'].values[0] if not call_df.empty else None
    put_price = put_df.iloc[(put_df['date'] - date).abs().argsort()[:1]]['close'].values[0] if not put_df.empty else None


    return call_price + put_price


### **EXECUTE**

In [ ]:
# Fetch options chains
# As it is not possible to fetch historical option chains
# fetch only options prices of last week with expiration date in 60 days from today

# Copy DataFrame
df_straddle_price = df_stock_price.copy()

# Get current date
current_date = datetime.today().strftime('%Y-%m-%d')

for ticker in df_stock_price.columns :
  for date in df_stock_price.index:

    # Ensure stock_price is a scalar value
    stock_price = df_stock_price.loc[date, ticker]

    if isinstance(stock_price, pd.Series):
        stock_price = stock_price.iloc[0] # Extract scalar if it's a Series

    # Fetch option chain
    dict_exp_strike = fetch_option_chain(ib, ticker, stock_price, current_date)

    # Ensure dict_exp_strike contains valid expirations
    if dict_exp_strike and 'expirations' in dict_exp_strike and len(dict_exp_strike['expirations']) > 2:
        exp = dict_exp_strike['expirations'][2]  # 60-day expiration if available
    else:
        exp = None

    # Get closest strike
    strike = dict_exp_strike['closest_strike']

    # Fetch straddle price
    straddle_price = fetch_intraday_straddle_prices(ib, ticker, exp, strike, date)

    # Store the price
    df_straddle_price.loc[date, ticker] = straddle_price

In [ ]:
# Process Nan
df_straddle_price.isna().sum()
df_straddle_price_cleaned = df_straddle_price.dropna(axis=1, thresh=0.4*len(df_straddle_price))
df_straddle_price_cleaned = df_straddle_price_cleaned.interpolate(method="linear", limit_direction="both")

In [ ]:
# Calculate straddle return
df_straddle_price = df_straddle_price_cleaned
df_straddle_return = np.log(df_straddle_price / df_straddle_price.shift(1))

## **CROSS SECTIONAL REGRESSION**

In [ ]:
# Ensure df is a DataFrame
df = df_straddle_return.copy()
df.index = pd.to_datetime(df.index)

# Define max lag for autocorrelation analysis
max_lag = 26

# Store regression results
regression_results = []

for k in range(1, max_lag + 1):  # Iterate over different lags
    df_lagged = df.shift(k)  # Create lagged data

    # Merge current and lagged data
    combined_data = pd.concat([df, df_lagged.rename(columns=lambda x: f"{x}_lag")], axis=1).dropna()

    if combined_data.empty:
        continue  # Skip iteration if no valid data

    # Perform regression for each column (cross-sectional)
    for ticker in df.columns:
        X = combined_data[f"{ticker}_lag"]
        y = combined_data[ticker]
        X = sm.add_constant(X)  # Add intercept

        model = sm.OLS(y, X).fit()

        # Store results (only gamma and t-stat)
        regression_results.append({
            'lag': k,
            'gamma': model.params.get(f"{ticker}_lag", np.nan),  # Coefficient
            'gamma_tstat': model.tvalues.get(f"{ticker}_lag", np.nan)  # T-statistic
        })

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(regression_results)

# Group results by lag and compute mean gamma and t-statistics
grouped_results = results_df.groupby('lag').mean()

In [ ]:
# Plot gamma coefficients vs lag
plt.figure(figsize=(10, 5))
plt.plot(grouped_results.index, grouped_results['gamma'], marker='o', linestyle='-', label="Gamma Coefficient")
plt.axhline(0, color='black', linestyle='--', linewidth=1)  # Reference line at 0
plt.xlabel("Lag (k)")
plt.ylabel("Gamma Coefficient")
plt.title("Gamma Coefficient vs Lag")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot t-statistics vs lag
plt.figure(figsize=(10, 5))
plt.plot(grouped_results.index, grouped_results['gamma_tstat'], marker='o', linestyle='-', color='red', label="Gamma T-Statistic")
plt.axhline(0, color='black', linestyle='--', linewidth=1)  # Reference line at 0
plt.xlabel("Lag (k)")
plt.ylabel("Gamma T-Statistic")
plt.title("T-Statistic of Gamma vs Lag")
plt.legend()
plt.grid(True)
plt.show()


## **BACKTEST H-L PORTFOLIO**

In [ ]:
# Ensure timestamp index is timezone-naive for consistency
df_log_returns = df_straddle_return.copy()  # Avoid modifying the original data
df_log_returns.index = pd.to_datetime(df_log_returns.index).tz_localize(None)

# Define the 13 30-minute timestamps per day
required_times = [
    "10:00", "10:30", "11:00", "11:30", "12:00", "12:30",
    "13:00", "13:30", "14:00", "14:30", "15:00", "15:30", "16:00"
]

# Dictionary to store strategy results for each 30-min period
dict_return_per_period = {}

for time in required_times:
    # Filter data for the specific time period each day
    df_filtered = df_log_returns[df_log_returns.index.strftime('%H:%M') == time]

    # Shift returns to get yesterday's returns at the same time period
    df_yesterday_returns = df_filtered.shift(1)

    # Dictionary to store daily strategy results for this period
    dict_return_per_day = {}

    for date in df_filtered.index[1:]:  # Start from second row to match shift(1)
        # Get yesterday's returns for ranking
        return_yst = df_yesterday_returns.loc[date].dropna()

        # Rank tickers into 5 quantiles
        quantiles = return_yst.rank(method="first", pct=True)  # Ranks from 0 to 1
        q1_tickers = quantiles[quantiles <= 0.2].index.tolist()  # Bottom 20% (short)
        q5_tickers = quantiles[quantiles >= 0.8].index.tolist()  # Top 20% (long)

        # Get today's returns for these tickers
        today_returns = df_filtered.loc[date]

        # Ensure tickers exist in `today_returns`
        q1_tickers = list(set(q1_tickers).intersection(today_returns.index))
        q5_tickers = list(set(q5_tickers).intersection(today_returns.index))

        # Store strategy returns
        dict_return_per_day[date] = {
            "Q1_tickers": q1_tickers,  # Short
            "Q5_tickers": q5_tickers,  # Long
            "Q1_returns": today_returns.loc[q1_tickers].sum() if q1_tickers else 0,
            "Q5_returns": today_returns.loc[q5_tickers].sum() if q5_tickers else 0,
            "H-L Return": (
                today_returns.loc[q5_tickers].sum() - today_returns.loc[q1_tickers].sum()
                if q1_tickers and q5_tickers
                else 0
            )
        }

    # Convert to DataFrame
    df_returns = pd.DataFrame.from_dict(dict_return_per_day, orient="index")

    # Store in dictionary
    dict_return_per_period[time] = df_returns


In [ ]:
plt.figure(figsize=(12, 6))

for time, df_returns in dict_return_per_period.items():
    if not df_returns.empty and "H-L Return" in df_returns.columns:
        # Ensure "H-L Return" is numeric
        df_returns["H-L Return"] = pd.to_numeric(df_returns["H-L Return"], errors="coerce")

        # Convert log-returns to cumulative return correctly using np.exp
        df_returns["Cumulative Return"] = (df_returns["H-L Return"].cumsum()).apply(np.exp) - 1

        # Explicitly update dictionary to avoid modifying the original reference
        dict_return_per_period[time] = df_returns

        # Plot the cumulative return
        plt.plot(df_returns.index, df_returns["Cumulative Return"], label=f"{time}")

plt.legend(title="30-Min Portfolios", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.title("Cumulative Returns of 30-Min Strategy Across Different Portfolios")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.grid()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))

sharpe_ratios = {}  # Dictionary to store Sharpe ratios for each portfolio

for time, df_returns in dict_return_per_period.items():
    if not df_returns.empty and "H-L Return" in df_returns.columns:
        # Ensure "H-L Return" is numeric
        df_returns["H-L Return"] = pd.to_numeric(df_returns["H-L Return"], errors="coerce")

        # Compute Cumulative Return
        df_returns["Cumulative Return"] = (df_returns["H-L Return"].cumsum()).apply(np.exp) - 1

        # Explicitly update dictionary to avoid modifying the original reference
        dict_return_per_period[time] = df_returns

        # Compute Sharpe Ratio
        mean_return = df_returns["H-L Return"].mean()  # Mean return
        std_return = df_returns["H-L Return"].std()  # Standard deviation
        sharpe_ratio = mean_return / std_return if std_return != 0 else np.nan  # Avoid division by zero
        sharpe_ratios[time] = sharpe_ratio  # Store in dictionary

        # Plot the cumulative return
        plt.plot(df_returns.index, df_returns["Cumulative Return"], label=f"{time} (Sharpe: {sharpe_ratio:.2f})")

plt.legend(title="30-Min Portfolios", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.title("Cumulative Returns of 30-Min Strategy Across Different Portfolios")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.grid()
plt.show()

# Display Sharpe Ratios
print("\n📊 **Sharpe Ratios for Each 30-Min Portfolio:**\n")
for time, ratio in sharpe_ratios.items():
    print(f"{time}: Sharpe Ratio = {ratio:.2f}")
